In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re 
import urllib
from urllib.request import urlopen
import time
import random

def writew(lst,file_):
    with open(file_,'w') as f:
        for l in lst:
            f.write(l)
            f.write('\n')
    
def writea(lst,file_):
    with open(file_,'a') as f:
        for l in lst:
            f.write(l)
            f.write('\n')
#create a webdriver object and set options for headless browsing
options = Options()
options.headless = True
browser = webdriver.Chrome('./chromedriver',options=options)

#uses webdriver object to execute javascript code and get dynamically loaded webcontent
def get_js_soup(url,browser):
    browser.get(url)
    res_html = browser.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

def scrape_page(dir_url,browser):
    links = []
    #execute js on webpage to load listings on webpage and get ready to parse the loaded HTML 
    soup = BeautifulSoup(dir_url,'html.parser')
    #soup = get_js_soup(dir_url,browser)
    for link_holder in soup.find_all('a',class_='v-align-middle'):
          try:
              rel_link = link_holder['href'] #get url
              links.append(rel_link)
          except:
              pass

    return links

links = []
for page in range(1,101):
    #dir_url = 'https://github.com/search?o=desc&p='+str(page)+'&q=react&s=stars&type=Repositories' #base url
    dir_url = "./html/"+str(page)+".html"
    ls = scrape_page(open(dir_url,'rb').read(),browser)
    for l in ls:
        links.append(l)
urls_file = 'urls.txt'
writew(links,urls_file)
'''
page = 1
while page < 100:
    url = 'https://github.com/search?o=desc&p='+str(page)+'&q=react&s=stars&type=Repositories'
    print(page)
    try:
        webContent = urlopen(url).read()
        ls = []
        #execute js on webpage to load listings on webpage and get ready to parse the loaded HTML 
        soup = BeautifulSoup(webContent,'html.parser')
        for link_holder in soup.find_all('a',class_='v-align-middle'):
              try:
                  rel_link = link_holder['href'] #get url
                  ls.append('https://github.com'+rel_link)
              except:
                  pass
        for l in ls:
            links.append(l)
        time.sleep(1)
        page += 1     
    except:
        time.sleep(60+random.random())

urls_file = 'urls.txt'
writew(links,urls_file)
'''

'''
#tidies extracted text 
def process_bio(bio):
    bio = bio.encode('ascii',errors='ignore').decode('utf-8')       #removes non-ascii characters
    bio = re.sub('\s+',' ',bio)       #repalces repeated whitespace characters with single space
    return bio

def remove_script(soup):
    for script in soup(["script", "style"]):
        script.decompose()
    return soup


#Checks if bio_url is valid 
def is_valid_homepage(bio_url,dir_url):
    try:
        ret_url = urllib.request.urlopen(bio_url).geturl() 
    except:
        return False
    urls = [re.sub('((https?://)|(www.))','',url) for url in [ret_url,dir_url]] #removes url scheme (https,http) or www 
    return not(urls[0]== urls[1])
    
'''

'\n#tidies extracted text \ndef process_bio(bio):\n    bio = bio.encode(\'ascii\',errors=\'ignore\').decode(\'utf-8\')       #removes non-ascii characters\n    bio = re.sub(\'\\s+\',\' \',bio)       #repalces repeated whitespace characters with single space\n    return bio\n\ndef remove_script(soup):\n    for script in soup(["script", "style"]):\n        script.decompose()\n    return soup\n\n\n#Checks if bio_url is valid \ndef is_valid_homepage(bio_url,dir_url):\n    try:\n        ret_url = urllib.request.urlopen(bio_url).geturl() \n    except:\n        return False\n    urls = [re.sub(\'((https?://)|(www.))\',\'\',url) for url in [ret_url,dir_url]] #removes url scheme (https,http) or www \n    return not(urls[0]== urls[1])\n    \n'

In [8]:
def scrape_faculty_page(url,browser):
    packages = []
    flag = False
    soup = get_js_soup(url+'/blob/master/package.json',browser)
    table = soup.find('table',class_='highlight tab-size js-file-line-container')
    try:
        for td in table.find_all('td',class_='blob-code blob-code-inner js-file-line'):
            if flag == True and '}' in td.text.lower():
                flag = False
            if flag == True:
                preparse = td.text
                postparse = ""
                isname = 0
                for s in preparse:
                    if s == '"' and isname == 0:
                        isname = 1
                    elif s == '"' and isname == 1:
                        isname = 2
                    elif isname == 1:
                        postparse += s
                packages.append(postparse)
            if isinstance(td.text, str) and 'dependencies' in td.text.lower():
                flag = True
    except:
        pass
    return packages

data = []
for link in links:
    package = scrape_faculty_page(link,browser)
    if len(package) > 0:
        data.append(package)

datatxt = []
for i in range(len(data)):
    datastr = '['
    for d in data[i]:
        datastr += d + ','
    datastr = datastr[:-1]
    datastr += ']'
    datatxt.append(datastr)

    
data_file = 'data.txt'
writew(datatxt,data_file)